In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Bidirectional, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot, LeakyReLU, GRU
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from keras import backend as K
import tensorflow as tf

import pydot as pyd
from keras.utils.vis_utils import plot_model, model_to_dot
keras.utils.vis_utils.pydot = pyd

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
  
# dataset_filename = 'dataset32.pkl'
# if (dataset_filename == 'dataset64.pkl'):
#   tf.keras.backend.set_floatx('float64')
# else:
#   tf.keras.backend.set_floatx('float32')

dataset_name = "MAESTRO"
# dataset_name = "GiantMIDIPiano"
# dataset_name = "chien2021"

N_HIDDEN = 4
dataset_filename = ''
if dataset_name == 'MAESTRO':
  dataset_filename = f'dataset32-{dataset_name}-len{N_HIDDEN}.pkl'
elif dataset_name == 'GiantMIDIPiano':
  dataset_filename = f'dataset32-{dataset_name}-len{N_HIDDEN}.pkl'
elif dataset_name == 'chien2021':
  dataset_filename = f'dataset32-{dataset_name}-len{N_HIDDEN}.pkl'

In [ ]:
data = pkl.load(open(dataset_filename, 'rb'))

dataset_train_input = data['dataset_train_input']
dataset_train_label = data['dataset_train_label']
if (dataset_name != 'chien2021'):
    dataset_val_input = data['dataset_val_input']
    dataset_val_label = data['dataset_val_label']
dataset_test_input = data['dataset_test_input']
dataset_test_label = data['dataset_test_label']

velocity_min = data['velocity_min']
velocity_max = data['velocity_max']


In [ ]:
input_train = Input(shape=(dataset_train_input.shape[1], dataset_train_input.shape[2]))
label_train = Input(shape=(dataset_train_label.shape[1], dataset_train_label.shape[2]))

print(input_train)
print(label_train)

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="midi-velocity-infer-v2-attention",

    # track hyperparameters and run metadata with wandb.config
    config={
        "n_hidden": N_HIDDEN,
        "activation_1": "LeakyRelu",
        "dropout": 0.2,
        "optimizer": "adam",
        "loss": "mse_cosine_loss",
        "loss_alpha": 0.15,
        "batchnorm_momentum": 0.60,
        "metric": "mae",
        "epoch": 5,
        "teacher_forcing": False,
        "rnn_type": "LSTM",
        "dataset_name": dataset_name
    }
)
config = wandb.config

In [ ]:
if config.rnn_type == "LSTM":
    encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
        config.n_hidden, activation=LeakyReLU(),
        input_shape=(dataset_train_input.shape[1], dataset_train_input.shape[2]), 
        return_sequences=True, return_state=True
    )(input_train)
    print(encoder_stack_h)
    print(encoder_last_h)
    print(encoder_last_c)
elif config.rnn_type == "GRU":
    encoder_stack_h, encoder_last_h = GRU(
    config.n_hidden, activation=LeakyReLU(),
    input_shape=(dataset_train_input.shape[1], dataset_train_input.shape[2]), 
    return_sequences=True, return_state=True
    )(input_train)
    print(encoder_stack_h)
    print(encoder_last_h)

In [ ]:
encoder_last_h = BatchNormalization(momentum=config.batchnorm_momentum)(encoder_last_h) 
print(encoder_last_h)
if config.rnn_type == "LSTM":
    encoder_last_c = BatchNormalization(momentum=config.batchnorm_momentum)(encoder_last_c) 
    print(encoder_last_c)
decoder_input = RepeatVector(dataset_train_input.shape[1])(encoder_last_h)
print(decoder_input)

In [ ]:
if config.rnn_type == "LSTM":
    decoder_stack_h = LSTM(
        N_HIDDEN, activation=LeakyReLU(), dropout=config.dropout,
        return_sequences=True, return_state=False
    )(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
elif config.rnn_type == "GRU":
    decoder_stack_h = GRU(
    N_HIDDEN, activation=LeakyReLU(), dropout=config.dropout,
    return_sequences=True, return_state=False
    )(decoder_input, initial_state=[encoder_last_h])

print(decoder_stack_h)

In [ ]:
attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
attention = Activation('softmax')(attention)
print(attention)

In [ ]:
context = dot([attention, encoder_stack_h], axes=[2, 1])
context = BatchNormalization(momentum=config.batchnorm_momentum)(context)
print(context)

In [ ]:
decoder_combined_context = concatenate([context, decoder_stack_h])
print(decoder_combined_context)

In [ ]:
out = TimeDistributed(Dense(dataset_train_label.shape[2]))(decoder_combined_context)
print(out)

In [ ]:
model = Model(inputs=input_train, outputs=out)
# step_size = 176787
lr_decay_alpha = 1.0
cosine_decay_annealing_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=0.0001, first_decay_steps=10000, alpha=0, t_mul=2, m_mul=0.9
)

if (dataset_name == 'chien2021'):
    cosine_decay_scheduler = tf.keras.optimizers.schedules.CosineDecay(
        # initial_learning_rate=0.0001, decay_steps=176787*config.epoch, alpha=0.001 # MAESTRO full
        initial_learning_rate=0.0001, decay_steps=31219*config.epoch*lr_decay_alpha, alpha=0.001 # chien2021
        # initial_learning_rate=0.0001, decay_steps=1889372, alpha=0.7 # GiantMIDIPiano, epoch4
    )
elif (dataset_name == 'MAESTRO'):
    cosine_decay_scheduler = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=0.0001, decay_steps=176787*config.epoch*lr_decay_alpha, alpha=0.001 # MAESTRO full len4
        # initial_learning_rate=0.0001, decay_steps=176670*config.epoch*lr_decay_alpha, alpha=0.001 # MAESTRO full len8
        # initial_learning_rate=0.0001, decay_steps=176432*config.epoch*lr_decay_alpha, alpha=0.001 # MAESTRO full len16
        # initial_learning_rate=0.0001, decay_steps=175952*config.epoch*lr_decay_alpha, alpha=0.001 # MAESTRO full len32
    )
elif (dataset_name == 'GiantMIDIPiano'):
        cosine_decay_scheduler = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=0.0001, decay_steps=1889372, alpha=0.7 # GiantMIDIPiano, epoch4
    )

step_decay_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.0001,
    decay_steps=95400,
    end_learning_rate=0.000001,
    power = 1.0
)

# opt = Adam(learning_rate=cosine_decay_annealing_scheduler)
opt = Adam(learning_rate=cosine_decay_scheduler)
# opt = Adam(learning_rate=step_decay_scheduler)
# opt = Adam(lr=0.0001, clip_norm=1.0, clipvalue=0.5)

from keras.losses import mse, cosine_similarity, mae
def make_mse_cosine_loss(alpha):
    def mse_cosine_loss(y_true, y_pred):
        # y_pred = tf.clip_by_value(y_pred, clip_value_min=0, clip_value_max=127)
        return alpha * (1 * cosine_similarity(y_true, y_pred)) + (1 - alpha) * mse(y_true, y_pred)
    return mse_cosine_loss
ALPHA = config.loss_alpha
mse_cosine_loss = make_mse_cosine_loss(ALPHA)

def clipped_loss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, clip_value_min=0, clip_value_max=127)
    loss = tf.losses.mean_squared_error(y_true, y_pred)
    return loss

model.compile(loss=mse_cosine_loss, optimizer=opt, metrics=['mae'])
model.summary()

In [ ]:
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
from datetime import datetime
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M_%S')
print(f'Current time: {current_time}')

import os
os.makedirs(f'saved_models', exist_ok=True)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging
tf.get_logger().setLevel(logging.ERROR) # TODO: comment if you need to debug

# es = EarlyStopping(monitor='val_loss', patience=10)
# history = model.fit(dataset_train_input, dataset_train_label, epochs=epoch, validation_data=(dataset_val_input, dataset_val_label), callbacks=[es])

if (dataset_name == 'chien2021'):
    history = model.fit(dataset_train_input, dataset_train_label, epochs=config.epoch, 
                        callbacks=[WandbMetricsLogger(log_freq=5), WandbModelCheckpoint("models")])
else:
    history = model.fit(dataset_train_input, dataset_train_label, epochs=config.epoch, validation_data=(dataset_val_input, dataset_val_label)
                        , callbacks=[WandbMetricsLogger(log_freq=5), WandbModelCheckpoint("models")])
model.save(f'saved_models/mvi-v2-{current_time}-h{config.n_hidden}-e{config.epoch}-{config.loss}-alpha{config.loss_alpha:.2f}-m{config.batchnorm_momentum:.2f}-{config.rnn_type}-luong_attention-{config.dataset_name}.h5')

In [ ]:
# model.save(f'saved_models/mvi-v2-{current_time}-h{config.n_hidden}-e{config.epoch}-{config.loss}-alpha{config.loss_alpha:.2f}-{config.rnn_type}-luong_attention-{config.dataset_name}.h5')

In [ ]:
wandb.alert(title='Training finished', text=f'loss: {history.history["loss"][-1]:.4f}, mae: {history.history["mae"][-1]:.4f}', level=wandb.AlertLevel.INFO)

In [ ]:
wandb.finish()

In [ ]:
if (dataset_name == 'chien2021'):
    train_mae = history.history['mae']
    
    plt.plot(train_mae, label='train mae')
    plt.ylabel('MAE')
    plt.xlabel('epoch')
    plt.title('MAE(train)')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=2)
    plt.show()
else:
    train_mae = history.history['mae']
    valid_mae = history.history['val_mae']

    plt.plot(train_mae, label='train mae'), 
    plt.plot(valid_mae, label='validation mae')
    plt.ylabel('mae')
    plt.xlabel('epoch')
    plt.title('train vs. validation accuracy (mae)')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=False, ncol=2)
    plt.show()